In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import math 
import collections
from pycocotools.coco import COCO
import requests
import plotly.express as px
import plotly.graph_objects as go
from os import listdir
from os.path import isfile, join
import base64
import itertools
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)
sns.set(style='whitegrid', font_scale=1.6, font='Georgia', context='paper')

from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import operator
import plotly.figure_factory as ff
import scipy
import pickle 
from sklearn.neighbors import KernelDensity
from tqdm import tqdm
from collections import OrderedDict

/tmp/ipykernel_22319/1341984599.py:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
sampling_base = '../../../src/dataset_selection/sampling/samples/LXR111/multilabel_full/'
base_path = '../../../snap/vqa/lxr111_multilabel_full_run_3/'
#sampling_path = 'beta/beta_kernel/tophat/seed_965/alpha_2_beta_1_budget_30.pkl'
sampling_path = 'beta/beta_pvals/seed_965/alpha_1_beta_2_budget_30.pkl'
df = pd.read_pickle(base_path+"datamap_metrics.pkl")

with open(sampling_base + sampling_path, 'rb') as f:
    sampled_ids = pickle.load(f)

df_sampled = df.loc[df['question_id'].isin(sampled_ids)] 



In [45]:
all_questions = df['Question'].tolist()

all_questions_preprocessed = []

for quest in all_questions:
    all_questions_preprocessed.append(quest[0].lower().split()[:4]) # crop questions by their first four words

In [70]:
first_word_freq = {}
for i in all_questions_preprocessed:
    if "'" in i[0]:
        contraction = i[0].index("'")
        first_word = i[0][:contraction]
    else:
        first_word = i[0]
    if first_word not in first_word_freq:
        first_word_freq[first_word] = 1
    else:
        first_word_freq[first_word] += 1

top_stems = dict(sorted(first_word_freq.items(), key=lambda x:x[1], reverse=True)[:10])

subset_questions = []
for stem in top_stems:
    for i in all_questions_preprocessed:
        #if i[0].startswith(stem):
        if "'" in i[0]:
            contraction = i[0].index("'")
            i[0] = i[0][:contraction]
        if i[0]==stem:
            subset_questions.append(i)

In [71]:
# for each stem in top_stems, create a subgroup for the second word
# top_stems
second_freq = {}
third_freq = {}
fourth_freq = {}

for i in subset_questions:
    first_two_words = ' '.join(i[:2])
    first_three_words = ' '.join(i[:3])
    first_four_words = ' '.join(i)

    if first_two_words not in second_freq:
        second_freq[first_two_words] = 1
    else:
        second_freq[first_two_words] += 1

    if first_three_words not in third_freq:
        third_freq[first_three_words] = 1
    else:
        third_freq[first_three_words] += 1
    if first_four_words not in fourth_freq:
        fourth_freq[first_four_words] = 1
    else:
        fourth_freq[first_four_words] += 1


In [80]:
# create list of colors
a, b, c, d, e, f, g, h, i, j =[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens, plt.cm.Greys, plt.cm.Purples,
                                plt.cm.Oranges, plt.cm.YlOrBr, plt.cm.YlOrRd, plt.cm.PuBuGn, plt.cm.YlGn]
colors = [a, b, c, d, e, f, g, h, i, j]
top_stems_keys = list(top_stems.keys())

second_colors = []
third_colors = []
for idx, stem in enumerate(top_stems_keys):
    for second_stem in second_freq:
        if second_stem.startswith(stem):
            second_colors.append(colors[idx](0.4))
    for third_stem in third_freq:
        if third_stem.startswith(stem):
            third_colors.append(colors[idx](0.4))





In [89]:
%matplotlib inline
plt.rcParams['figure.figsize'] = [40, 40]

stem_names = list(top_stems.keys())
stem_freqs = list(top_stems.values())

second_names=list(second_freq.keys())
subgroup_names = []
for second in second_names:
    subgroup_names.append(second.split()[1])
subgroup_size=list(second_freq.values())

# third_names=list(third_freq.keys())
# subgroup_names_three = []
# for third in third_names:
#     subgroup_names_three.append(third.split()[2])
# subgroup_size_three=list(third_freq.values())


#a, b, c, d, e, f, g, h, i, j =[plt.cm.Blues, plt.cm.Reds, plt.cm.Greens, plt.cm.Greys, plt.cm.Purples,
#                                plt.cm.Oranges, plt.cm.YlOrBr, plt.cm.YlOrRd, plt.cm.PuBuGn, plt.cm.YlGn]
# First Ring (outside)
fig, ax = plt.subplots()
ax.axis('equal')


# mypie2, _ = ax.pie(subgroup_size_three, radius=1.3, labels=subgroup_names_three, colors=third_colors, rotatelabels=True, textprops={'fontsize': 6})
# plt.setp( mypie2, width=0.4, edgecolor='white')

# Second Ring (Inside)
mypie2, _ = ax.pie(subgroup_size, radius=1.3, labels=subgroup_names, colors=second_colors, rotatelabels=True, textprops={'fontsize': 10})
plt.setp( mypie2, width=0.4, edgecolor='white')

mypie, _ = ax.pie(stem_freqs, radius=1.3-0.3, labels=stem_names, labeldistance=0.7, colors=[a(0.6), b(0.6), c(0.6), d(0.6), e(0.6), 
                                                                        f(0.6), g(0.6), h(0.6), i(0.6), j(0.6)], rotatelabels=True, textprops={'fontsize': 12})
plt.setp( mypie, width=0.3, edgecolor='white')


plt.margins(0,0)



plt.show()

findfont: Font family ['Georgia'] not found. Falling back to DejaVu Sans.
findfont: Font family ['Georgia'] not found. Falling back to DejaVu Sans.
